<a href="https://colab.research.google.com/github/Drisya-Ponmari/UST_INTERN/blob/master/%20/Webapp/backend.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#import flasgger
#from flasgger import Swagger
from flask import Flask,request
from transformers import AutoTokenizer,AutoModelForQuestionAnswering
from flask_ngrok import run_with_ngrok
import torch
app=Flask(__name__)
#Swagger(app)
run_with_ngrok(app) 

@app.route('/')
def welcome():
	return "Welcome People"

#Extracting the input from the request
@app.route('/answer',methods=["GET","POST"])
def Answering():
	Inputs = request.get_json()
	print(Inputs)
	Question = Inputs['question']
	Answer   = Inputs['context']

	#model and tokeniser from Hugging face
	tokenizer =  AutoTokenizer.from_pretrained('ponmari/Question-Answering')
	model = AutoModelForQuestionAnswering.from_pretrained('ponmari/Question-Answering')

	
	input_ids = tokenizer.encode(Question,Answer)
	tokens = tokenizer.convert_ids_to_tokens(input_ids)
	sep_index = input_ids.index(tokenizer.sep_token_id)

	num_seg_a = sep_index+1
	num_seg_b = len(input_ids) - num_seg_a

	segment_ids = [0]*num_seg_a + [1]*num_seg_b

	assert len(segment_ids) == len(input_ids)

	#Running the model with input
	start_scores,end_scores = model(torch.tensor([input_ids]),token_type_ids=torch.tensor([segment_ids]))

	answer_start = torch.argmax(start_scores)
	answer_end = torch.argmax(end_scores)

	
  
	# Start with the first token.
	answer = tokens[answer_start]

# Select the remaining answer tokens and join them with whitespace.
	for i in range(answer_start + 1, answer_end + 1):
    
    # If it's a subword token, then recombine it with the previous token.
		if tokens[i][0:2] == '##':
			answer += tokens[i][2:]
    
    # Otherwise, add a space then the token.
		else:
			answer += ' ' + tokens[i]
	
	print(answer)
	return {'Aswer':answer}
@app.route('/test',methods=["GET","POST"])
def Test():
	print ("Runnig")
	I = request.get_json()
	print(I)
	return {'data':1}
if __name__ == "__main__":
	app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://d9af096e71bf.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [24/Jul/2020 06:44:53] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [24/Jul/2020 06:44:54] "GET /favicon.ico HTTP/1.1" 404 -


{'question': 'What did Drisya love', 'context': 'Drisya loves pizza'}


[2020-07-24 06:46:35,789] ERROR in app: Exception on /answer [POST]
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/lib/python3.6/dist-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/lib/python3.6/dist-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/usr/local/lib/python3.6/dist-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/usr/local/lib/python3.6/dist-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/lib/python3.6/dist-packages/flask/app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "<ipython-input-5-7088247bf2fa>", line 40, in Answering
    start_scores,end_scores = model(torch.te

In [ ]:
!pip install transformers

In [ ]:
!pip install flask_ngrok